In [5]:
import numpy as np
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:70% !important; }</style>"))


/var/folders/lj/2t8twtsx5v1dw1dkbr59j1xr0000gn/T/ipykernel_30494/3431412320.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [9]:
## NLF Playoff Model
# Modeling EV Delta for each respective matchup
# Concept: Redistributing losing team's portion of the pie to the remaining playoff teams. Intra-conference teams receive greater redistribution share


import copy

# Define the probabilities and payouts for each team in the AFC and NFC
teams_probs = [
    # AFC Teams
    {'team': 'Team 1', 'seed': 1, 'conference': 'AFC', 'win_conference': 0.30, 'win_championship': 0.18, 'payout_conference': 5000, 'payout_championship': 20000},
    {'team': 'Team 2', 'seed': 2, 'conference': 'AFC', 'win_conference': 0.25, 'win_championship': 0.15, 'payout_conference': 6000, 'payout_championship': 25000},
    {'team': 'Team 3', 'seed': 3, 'conference': 'AFC', 'win_conference': 0.15, 'win_championship': 0.08, 'payout_conference': 7000, 'payout_championship': 30000},
    {'team': 'Team 4', 'seed': 4, 'conference': 'AFC', 'win_conference': 0.10, 'win_championship': 0.05, 'payout_conference': 8000, 'payout_championship': 35000},
    {'team': 'Team 5', 'seed': 5, 'conference': 'AFC', 'win_conference': 0.08, 'win_championship': 0.04, 'payout_conference': 9000, 'payout_championship': 40000},
    {'team': 'Team 6', 'seed': 6, 'conference': 'AFC', 'win_conference': 0.07, 'win_championship': 0.03, 'payout_conference': 10000, 'payout_championship': 45000},
    {'team': 'Team 7', 'seed': 7, 'conference': 'AFC', 'win_conference': 0.05, 'win_championship': 0.02, 'payout_conference': 12000, 'payout_championship': 50000},
    # NFC Teams
    {'team': 'Team 8', 'seed': 8, 'conference': 'NFC', 'win_conference': 0.28, 'win_championship': 0.15, 'payout_conference': 5000, 'payout_championship': 20000},
    {'team': 'Team 9', 'seed': 9, 'conference': 'NFC', 'win_conference': 0.22, 'win_championship': 0.12, 'payout_conference': 6000, 'payout_championship': 25000},
    {'team': 'Team 10', 'seed': 10, 'conference': 'NFC', 'win_conference': 0.15, 'win_championship': 0.07, 'payout_conference': 7000, 'payout_championship': 30000},
    {'team': 'Team 11', 'seed': 11, 'conference': 'NFC', 'win_conference': 0.12, 'win_championship': 0.06, 'payout_conference': 8000, 'payout_championship': 35000},
    {'team': 'Team 12', 'seed': 12, 'conference': 'NFC', 'win_conference': 0.09, 'win_championship': 0.05, 'payout_conference': 9000, 'payout_championship': 40000},
    {'team': 'Team 13', 'seed': 13, 'conference': 'NFC', 'win_conference': 0.08, 'win_championship': 0.04, 'payout_conference': 10000, 'payout_championship': 45000},
    {'team': 'Team 14', 'seed': 14, 'conference': 'NFC', 'win_conference': 0.06, 'win_championship': 0.03, 'payout_conference': 12000, 'payout_championship': 50000}
]

# Define function to update win probabilities based on proximity weights
def update_win_probs_with_proximity(winning_team_seed, losing_team_seed, teams_probs, conference):
    # Find the losing team's probabilities
    losing_team = next(team for team in teams_probs if team['seed'] == losing_team_seed and team['conference'] == conference)
    
    # Find remaining teams in both conferences
    remaining_teams = [copy.deepcopy(team) for team in teams_probs if not (team['seed'] == losing_team_seed and team['conference'] == conference)]
    
    # Calculate total remaining probabilities for conference and championship
    total_conference_prob = sum(team['win_conference'] for team in remaining_teams if team['conference'] == conference)
    total_championship_prob_same_conference = sum(team['win_championship'] for team in remaining_teams if team['conference'] == conference)
    total_championship_prob_opposite_conference = sum(team['win_championship'] for team in remaining_teams if team['conference'] != conference)
    
    # Proximity weights
    same_conference_weight = 0.8  # Teams in the same conference as the winner for championship redistribution
    opposite_conference_weight = 0.2  # Teams in opposite conference for championship redistribution
    
    # Calculate total redistribution amounts for conference and championship
    redistribution_conference_amount = losing_team['win_conference']
    redistribution_championship_amount_same_conference = same_conference_weight * losing_team['win_championship']
    redistribution_championship_amount_opposite_conference = opposite_conference_weight * losing_team['win_championship']
    
    # Redistribute probabilities
    for team in remaining_teams:
        # Redistribute conference probabilities within same conference
        if team['conference'] == conference:
            # Proportional redistribution within same conference (100% redistribution)
            team['adjusted_win_conference'] = team['win_conference'] + \
                (team['win_conference'] / total_conference_prob) * redistribution_conference_amount
        else:
            # Conference probability for the opposite conference remains unchanged
            team['adjusted_win_conference'] = team['win_conference']
    
        # Redistribute championship probabilities
        if team['conference'] == conference and total_championship_prob_same_conference > 0:  # Same conference
            team['adjusted_win_championship'] = team['win_championship'] + \
                (team['win_championship'] / total_championship_prob_same_conference) * redistribution_championship_amount_same_conference
        elif team['conference'] != conference and total_championship_prob_opposite_conference > 0:  # Opposite conference
            team['adjusted_win_championship'] = team['win_championship'] + \
                (team['win_championship'] / total_championship_prob_opposite_conference) * redistribution_championship_amount_opposite_conference
        else:
            # If no redistribution happens cuz of zero probability, retain original probability
            team['adjusted_win_championship'] = team['win_championship']
    
    return remaining_teams


# Function to calculate total EV for conference and championship bets
def calculate_total_ev(remaining_teams):
    """
    Calculate the total expected value for all teams given updated probabilities.
    :param remaining_teams: List of teams with updated probabilities.
    :return: Total expected value of the portfolio.
    """
    ev_conference = sum(team['adjusted_win_conference'] * team['payout_conference'] for team in remaining_teams)
    ev_championship = sum(team['adjusted_win_championship'] * team['payout_championship'] for team in remaining_teams)
    return ev_conference + ev_championship

# Function to calculate EV delta for all first round matchups
def calculate_ev_deltas_for_all_matchups(teams_probs):
    # Matchups with seed labels for each team
    matchups = [
        # AFC matchups
        ('AFC', 2, 7), ('AFC', 3, 6), ('AFC', 4, 5),
        # NFC matchups
        ('NFC', 9, 14), ('NFC', 10, 13), ('NFC', 11, 12)
    ]
    
    ev_deltas = {}
    
    for conference, seed1, seed2 in matchups:
        # Deep copy original teams_probs for each scenario
        original_probs = copy.deepcopy(teams_probs)
        
        # Scenario 1: Higher seed wins
        updated_probs_higher_seed_wins = update_win_probs_with_proximity(seed1, seed2, original_probs, conference)
        ev_higher_seed_wins = calculate_total_ev(updated_probs_higher_seed_wins)
        
        # Deep copy the original teams_probs for the second scenario
        original_probs = copy.deepcopy(teams_probs)
        
        # Scenario 2: Lower seed wins
        updated_probs_lower_seed_wins = update_win_probs_with_proximity(seed2, seed1, original_probs, conference)
        ev_lower_seed_wins = calculate_total_ev(updated_probs_lower_seed_wins)
        
        # Calculate the EV delta
        ev_delta = abs(ev_higher_seed_wins - ev_lower_seed_wins)
        
        ev_deltas[(conference, seed1, seed2)] = ev_delta
    
    return ev_deltas

# Calculate and output the EV deltas for all first-round matchups
ev_deltas = calculate_ev_deltas_for_all_matchups(teams_probs)
for matchup, ev_delta in ev_deltas.items():
    conference, seed1, seed2 = matchup
    print(f"EV Delta for {conference} matchup between Seed {seed1} and Seed {seed2}: {ev_delta}")


EV Delta for AFC matchup between Seed 2 and Seed 7: 1658.7735849056553
EV Delta for AFC matchup between Seed 3 and Seed 6: 586.9791902823126
EV Delta for AFC matchup between Seed 4 and Seed 5: 196.25761252104712
EV Delta for NFC matchup between Seed 9 and Seed 14: 1934.215585832615
EV Delta for NFC matchup between Seed 10 and Seed 13: 889.0080859748414
EV Delta for NFC matchup between Seed 11 and Seed 12: 269.19283122057095


In [10]:
## MLB Playoff Model
# Modeling EV Delta for each respective matchup
# Includes queried Potential Payouts

import copy
import mysql.connector
from collections import defaultdict

# Establish connection to MySQL database
conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

#cursor = conn.cursor()

# Query to get Potential Payouts grouped by ParticipantName and EventType
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1 
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.EventType IN ('Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

#
cursor.execute(query)

#
payouts = defaultdict(lambda: {'payout_conference': 0, 'payout_championship': 0})

for participant_name, event_type, total_payout in cursor.fetchall():
    # 
    team_name = participant_name
    # Convert decimal to float
    total_payout = float(total_payout)
    
    if event_type == 'Conference Winner':
        payouts[team_name]['payout_conference'] = total_payout
    elif event_type == 'Championship':
        payouts[team_name]['payout_championship'] = total_payout

#cursor.close()
conn.close()

# Define the probabilities and payouts for each team in the AL and NL
teams_probs_mlb = [
    # AL Teams
    {'team': 'New York Yankees', 'seed': 1, 'conference': 'AL', 'win_conference': 0.34, 'win_championship': 0.17},
    {'team': 'Cleveland Guardians', 'seed': 2, 'conference': 'AL', 'win_conference': 0.30, 'win_championship': 0.15},
    {'team': 'Houston Astros', 'seed': 3, 'conference': 'AL', 'win_conference': 0.16, 'win_championship': 0.08},
    {'team': 'Baltimore Orioles', 'seed': 4, 'conference': 'AL', 'win_conference': 0.1, 'win_championship': 0.06},
    {'team': 'Kansas City Royals', 'seed': 5, 'conference': 'AL', 'win_conference': 0.04, 'win_championship': 0.02},
    {'team': 'Detroit Tigers', 'seed': 6, 'conference': 'AL', 'win_conference': 0.04, 'win_championship': 0.02},
    # NL Teams
    {'team': 'Los Angeles Dodgers', 'seed': 1, 'conference': 'NL', 'win_conference': 0.40, 'win_championship': 0.20},
    {'team': 'Philadelphia Phillies', 'seed': 2, 'conference': 'NL', 'win_conference': 0.30, 'win_championship': 0.15},
    {'team': 'Milwaukee Brewers', 'seed': 3, 'conference': 'NL', 'win_conference': 0.08, 'win_championship': 0.04},
    {'team': 'San Diego Padres', 'seed': 4, 'conference': 'NL', 'win_conference': 0.14, 'win_championship': 0.07},
    {'team': 'Arizona Diamondbacks', 'seed': 5, 'conference': 'NL', 'win_conference': 0.04, 'win_championship': 0.02},
    {'team': 'New York Mets', 'seed': 6, 'conference': 'NL', 'win_conference': 0.04, 'win_championship': 0.02}
]

# Update the teams_probs_mlb dictionary with the payouts from the database
for team in teams_probs_mlb:
    team_name = team['team']
    if team_name in payouts:
        team['payout_conference'] = float(payouts[team_name]['payout_conference'])
        team['payout_championship'] = float(payouts[team_name]['payout_championship'])
    else:
        team['payout_conference'] = 0.0
        team['payout_championship'] = 0.0

# Function to update win probabilities based on proximity weights for MLB
def update_win_probs_with_proximity_mlb(winning_team_seed, losing_team_seed, teams_probs, conference):
    """
    Update win probabilities based on the outcome of a specific matchup and proximity in the bracket.
    :param winning_team_seed: Seed of the team that won the matchup.
    :param losing_team_seed: Seed of the team that lost the matchup.
    :param teams_probs: List of dictionaries with each team's probabilities and seed information.
    :param conference: The conference (AL or NL) where the matchup took place.
    :return: Updated probabilities for all remaining teams.
    """
    # Find the losing team's probabilities
    losing_team = next(team for team in teams_probs if team['seed'] == losing_team_seed and team['conference'] == conference)
    
    # Find remaining teams in both conferences
    remaining_teams = [copy.deepcopy(team) for team in teams_probs if not (team['seed'] == losing_team_seed and team['conference'] == conference)]
    
    # Calculate the total remaining probabilities for conference and championship
    total_conference_prob = sum(team['win_conference'] for team in remaining_teams if team['conference'] == conference)
    total_championship_prob_same_conference = sum(team['win_championship'] for team in remaining_teams if team['conference'] == conference)
    total_championship_prob_opposite_conference = sum(team['win_championship'] for team in remaining_teams if team['conference'] != conference)
    
    # Proximity weights
    same_conference_weight = 0.8  # 80% to the same conference as the winner
    opposite_conference_weight = 0.2  # 20% to the opposite conference
    
    # Calculate the total redistribution amounts for conference and championship
    redistribution_conference_amount = losing_team['win_conference']
    redistribution_championship_amount_same_conference = same_conference_weight * losing_team['win_championship']
    redistribution_championship_amount_opposite_conference = opposite_conference_weight * losing_team['win_championship']
    
    # Redistribute probabilities
    for team in remaining_teams:
        # Redistribute conference probabilities within the same conference
        if team['conference'] == conference:
            team['adjusted_win_conference'] = team['win_conference'] + \
                (team['win_conference'] / total_conference_prob) * redistribution_conference_amount
        else:
            team['adjusted_win_conference'] = team['win_conference']
    
        # Redistribute championship probabilities
        if team['conference'] == conference and total_championship_prob_same_conference > 0:  # Same conference
            team['adjusted_win_championship'] = team['win_championship'] + \
                (team['win_championship'] / total_championship_prob_same_conference) * redistribution_championship_amount_same_conference
        elif team['conference'] != conference and total_championship_prob_opposite_conference > 0:  # Opposite conference
            team['adjusted_win_championship'] = team['win_championship'] + \
                (team['win_championship'] / total_championship_prob_opposite_conference) * redistribution_championship_amount_opposite_conference
        else:
            team['adjusted_win_championship'] = team['win_championship']
    
    return remaining_teams

# Function to calculate total EV for conference and championship bets for MLB
def calculate_total_ev_mlb(remaining_teams):
    """
    Calculate the total expected value for all teams given updated probabilities.
    :param remaining_teams: List of teams with updated probabilities.
    :return: Total expected value of the portfolio.
    """
    ev_conference = sum(team['adjusted_win_conference'] * team['payout_conference'] for team in remaining_teams)
    ev_championship = sum(team['adjusted_win_championship'] * team['payout_championship'] for team in remaining_teams)
    return ev_conference + ev_championship

# Function to calculate EV delta for all first-round matchups for MLB
def calculate_ev_deltas_for_all_matchups_mlb(teams_probs):
    # Define matchups for the wildcard round
    matchups = [
        # AL matchups
        ('AL', 3, 6), ('AL', 4, 5),
        # NL matchups
        ('NL', 3, 6), ('NL', 4, 5)
    ]
    
    ev_deltas = {}
    
    for conference, seed1, seed2 in matchups:
        original_probs = copy.deepcopy(teams_probs)
        
        # Scenario 1: Higher seed wins
        updated_probs_higher_seed_wins = update_win_probs_with_proximity_mlb(seed1, seed2, original_probs, conference)
        ev_higher_seed_wins = calculate_total_ev_mlb(updated_probs_higher_seed_wins)
        
        # Deep copy the original teams_probs for the second scenario
        original_probs = copy.deepcopy(teams_probs)
        
        # Scenario 2: Lower seed wins
        updated_probs_lower_seed_wins = update_win_probs_with_proximity_mlb(seed2, seed1, original_probs, conference)
        ev_lower_seed_wins = calculate_total_ev_mlb(updated_probs_lower_seed_wins)
        
        # Calculate the EV delta
        ev_delta = abs(ev_higher_seed_wins - ev_lower_seed_wins)
        
        ev_deltas[(conference, seed1, seed2)] = ev_delta
    
    return ev_deltas

# Calculate and output the EV deltas for all first-round matchups for MLB
ev_deltas_mlb = calculate_ev_deltas_for_all_matchups_mlb(teams_probs_mlb)
for matchup, ev_delta in ev_deltas_mlb.items():
    conference, seed1, seed2 = matchup
    print(f"EV Delta for {conference} matchup between Seed {seed1} and Seed {seed2}: {ev_delta}")


EV Delta for AL matchup between Seed 3 and Seed 6: 306.87811516544025
EV Delta for AL matchup between Seed 4 and Seed 5: 1442.5115388781342
EV Delta for NL matchup between Seed 3 and Seed 6: 236.16674695652182
EV Delta for NL matchup between Seed 4 and Seed 5: 2841.133597674416


In [14]:
# Define the probabilities and payouts for each team in the AFC and NFC
# Modeling EV Delta for a sinlge matchup
teams_probs = [
    # AFC Teams
    {'team': 'Team 1', 'seed': 1, 'conference': 'AFC', 'win_conference': 0.30, 'win_championship': 0.18, 'payout_conference': 5000, 'payout_championship': 20000},
    {'team': 'Team 2', 'seed': 2, 'conference': 'AFC', 'win_conference': 0.25, 'win_championship': 0.15, 'payout_conference': 6000, 'payout_championship': 25000},
    {'team': 'Team 3', 'seed': 3, 'conference': 'AFC', 'win_conference': 0.15, 'win_championship': 0.08, 'payout_conference': 7000, 'payout_championship': 30000},
    {'team': 'Team 4', 'seed': 4, 'conference': 'AFC', 'win_conference': 0.10, 'win_championship': 0.05, 'payout_conference': 8000, 'payout_championship': 35000},
    {'team': 'Team 5', 'seed': 5, 'conference': 'AFC', 'win_conference': 0.08, 'win_championship': 0.04, 'payout_conference': 9000, 'payout_championship': 40000},
    {'team': 'Team 6', 'seed': 6, 'conference': 'AFC', 'win_conference': 0.07, 'win_championship': 0.03, 'payout_conference': 10000, 'payout_championship': 45000},
    {'team': 'Team 7', 'seed': 7, 'conference': 'AFC', 'win_conference': 0.05, 'win_championship': 0.02, 'payout_conference': 12000, 'payout_championship': 50000},
    # NFC Teams
    {'team': 'Team 8', 'seed': 1, 'conference': 'NFC', 'win_conference': 0.28, 'win_championship': 0.15, 'payout_conference': 5000, 'payout_championship': 20000},
    {'team': 'Team 9', 'seed': 2, 'conference': 'NFC', 'win_conference': 0.22, 'win_championship': 0.12, 'payout_conference': 6000, 'payout_championship': 25000},
    {'team': 'Team 10', 'seed': 3, 'conference': 'NFC', 'win_conference': 0.15, 'win_championship': 0.07, 'payout_conference': 7000, 'payout_championship': 30000},
    {'team': 'Team 11', 'seed': 4, 'conference': 'NFC', 'win_conference': 0.12, 'win_championship': 0.06, 'payout_conference': 8000, 'payout_championship': 35000},
    {'team': 'Team 12', 'seed': 5, 'conference': 'NFC', 'win_conference': 0.09, 'win_championship': 0.05, 'payout_conference': 9000, 'payout_championship': 40000},
    {'team': 'Team 13', 'seed': 6, 'conference': 'NFC', 'win_conference': 0.08, 'win_championship': 0.04, 'payout_conference': 10000, 'payout_championship': 45000},
    {'team': 'Team 14', 'seed': 7, 'conference': 'NFC', 'win_conference': 0.06, 'win_championship': 0.03, 'payout_conference': 12000, 'payout_championship': 50000}
]

# Define function to update win probabilities based on proximity weights
def update_win_probs_with_proximity(winning_team_seed, teams_probs):
    """
    Update win probabilities based on the outcome of a specific matchup and proximity in the bracket.
    :param winning_team_seed: Seed of the team that won the matchup (2 or 7).
    :param teams_probs: List of dictionaries with each team's probabilities and seed information.
    :return: Updated probabilities for all remaining teams.
    """
    # Determine the losing team's probabilities
    losing_team_seed = 7 if winning_team_seed == 2 else 2
    losing_team = next(team for team in teams_probs if team['seed'] == losing_team_seed and team['conference'] == 'AFC')
    
    # Find remaining teams in both AFC and NFC
    remaining_teams = [team for team in teams_probs if not (team['seed'] == losing_team_seed and team['conference'] == 'AFC')]
    
    # Calculate the total remaining probabilities for conference and championship
    total_conference_prob = sum(team['win_conference'] for team in remaining_teams if team['conference'] == 'AFC')
    total_championship_prob = sum(team['win_championship'] for team in remaining_teams)
    
    # Proximity weights (the matchup's outcome has greater impact on teams within the same conference)
    same_conference_weight = 0.8  # Teams in the same conference as the winner
    opposite_conference_weight = 0.2  # Teams in the opposite conference
    
    for team in remaining_teams:
        # Determine proximity weight
        if team['conference'] == 'AFC':  # Same conference as the winner
            proximity_weight = same_conference_weight
        else:  # Opposite conference (NFC)
            proximity_weight = opposite_conference_weight

        # Calculate adjusted conference probability (only for AFC teams)
        if team['conference'] == 'AFC':
            redistribution_share_conference = (team['win_conference'] / total_conference_prob) * proximity_weight
            team['adjusted_win_conference'] = team['win_conference'] + (redistribution_share_conference * losing_team['win_conference'])
        else:
            team['adjusted_win_conference'] = team['win_conference']

        # Calculate adjusted championship probability (all teams)
        redistribution_share_championship = (team['win_championship'] / total_championship_prob) * proximity_weight
        team['adjusted_win_championship'] = team['win_championship'] + (redistribution_share_championship * losing_team['win_championship'])
    
    return remaining_teams

# Function to calculate total EV for conference and championship bets
def calculate_total_ev(remaining_teams):
    """
    Calculate the total expected value for all teams given updated probabilities.
    :param remaining_teams: List of teams with updated probabilities.
    :return: Total expected value of the portfolio.
    """
    ev_conference = sum(team['adjusted_win_conference'] * team['payout_conference'] for team in remaining_teams if team['conference'] == 'AFC')
    ev_championship = sum(team['adjusted_win_championship'] * team['payout_championship'] for team in remaining_teams)
    return ev_conference + ev_championship

# Scenario 1: Team 2 wins the matchup
updated_teams_probs_team2_wins = update_win_probs_with_proximity(2, teams_probs)
ev_team2_wins = calculate_total_ev(updated_teams_probs_team2_wins)

# Scenario 2: Team 7 wins the matchup
updated_teams_probs_team7_wins = update_win_probs_with_proximity(7, teams_probs)
ev_team7_wins = calculate_total_ev(updated_teams_probs_team7_wins)

# Calculate the EV delta
ev_delta = abs(ev_team2_wins - ev_team7_wins)

# Output the results
print(f"Expected value if Team 2 wins the matchup: {ev_team2_wins}")
print(f"Expected value if Team 7 wins the matchup: {ev_team7_wins}")
print(f"EV Delta between Team 2 and Team 7 winning: {ev_delta}")


Expected value if Team 2 wins the matchup: 36763.23809523809
Expected value if Team 7 wins the matchup: 36033.52173913043
EV Delta between Team 2 and Team 7 winning: 729.7163561076595


In [7]:
# Define function to calculate the conditional probability given a team wins the current round
def update_future_probs(prob_win_round, prob_win_future):
    """
    Update the probability of winning the future round (conference or championship)
    given that a team has already won the current round.
    :param prob_win_round: Probability of winning the current round
    :param prob_win_future: Probability of winning the future round before the current round is played
    :return: Updated probability of winning the future round if the current round is won
    """
    if prob_win_round == 0:  # Can't div /0
        return 0
    return prob_win_future / prob_win_round

# Define function to calculate the expected payout for a single bet
def calculate_expected_payout(prob_win, payout):
    """
    Calculate the expected payout for a bet.
    :param prob_win: Probability of winning the bet (conference or championship)
    :param payout: Potential payout if the bet wins
    :return: Expected payout
    """
    return prob_win * payout

# Define the main function to calculate EV for each scenario
def calculate_portfolio_ev(teamA_probs, teamB_probs, teamA_payouts, teamB_payouts):
    """
    Calculate the total expected value (expected payout) of the portfolio depending on Team A or Team B winning.
    :param teamA_probs: Pre-round probabilities for Team A (dictionary with keys: 'win_matchup', 'win_conference', 'win_championship')
    :param teamB_probs: Pre-round probabilities for Team B (same structure as teamA_probs)
    :param teamA_payouts: Payout information for Team A (dictionary with keys: 'payout_conference', 'payout_championship')
    :param teamB_payouts: Payout information for Team B (same structure as teamA_payouts)
    :return: EV for Team A wins and EV for Team B wins, and the delta
    """
    # Update future probabilities based on the outcome of the current round
    teamA_updated_probs = {
        'win_conference': update_future_probs(teamA_probs['win_matchup'], teamA_probs['win_conference']),
        'win_championship': update_future_probs(teamA_probs['win_matchup'], teamA_probs['win_championship'])
    }
    
    teamB_updated_probs = {
        'win_conference': update_future_probs(teamB_probs['win_matchup'], teamB_probs['win_conference']),
        'win_championship': update_future_probs(teamB_probs['win_matchup'], teamB_probs['win_championship'])
    }

    # Scenario 1: Team A wins the matchup
    ev_teamA_win = (
        teamA_updated_probs['win_conference'] * calculate_expected_payout(teamA_updated_probs['win_championship'], teamA_payouts['payout_championship']) +
        calculate_expected_payout(teamA_updated_probs['win_conference'], teamA_payouts['payout_conference'])
    )

    # Team B doesn't advance so no payout for their futures
    ev_teamB_lose = 0

    total_ev_teamA_wins = ev_teamA_win + ev_teamB_lose

    # Scenario 2: Team B wins the matchup
    ev_teamB_win = (
        teamB_updated_probs['win_conference'] * calculate_expected_payout(teamB_updated_probs['win_championship'], teamB_payouts['payout_championship']) +
        calculate_expected_payout(teamB_updated_probs['win_conference'], teamB_payouts['payout_conference'])
    )

    # Team A doesn't advance so no payout for their futures
    ev_teamA_lose = 0

    total_ev_teamB_wins = ev_teamB_win + ev_teamA_lose

    # Calculate the EV delta
    ev_delta = abs(total_ev_teamA_wins - total_ev_teamB_wins)

    return total_ev_teamA_wins, total_ev_teamB_wins, ev_delta

# Example input data: Musetti-Djokovic Wimbledon SF Matchup
teamA_probs = {
    'win_matchup': 0.13,            # Probability of Team A winning the current round
    'win_conference': 0.13,         # Probability of winning the conference prior to the current round
    'win_championship': 0.03        # Probability of winning the championship prior to the current round
}

teamB_probs = {
    'win_matchup': 0.87,           # Probability of Team B winning the current round
    'win_conference': 0.87,        # Probability of winning the conference prior to the current round
    'win_championship': 0.43       # Probability of winning the championship prior to the current round
}

teamA_payouts = {
    'payout_conference': 0,      # Payout if Team A wins the conference
    'payout_championship': 48072    # Payout if Team A wins the championship
}

teamB_payouts = {
    'payout_conference': 18886,      # Payout if Team B wins the conference
    'payout_championship': 63694     # Payout if Team B wins the championship
}

# Run the model
ev_teamA_wins, ev_teamB_wins, ev_delta = calculate_portfolio_ev(teamA_probs, teamB_probs, teamA_payouts, teamB_payouts)

# Output the results
print(f"Expected payout if Team A wins: {ev_teamA_wins}")
print(f"Expected payout if Team B wins: {ev_teamB_wins}")
print(f"Delta in expected payout: {ev_delta}")


Expected payout if Team A wins: 11093.538461538461
Expected payout if Team B wins: 50366.94252873563
Delta in expected payout: 39273.40406719717


In [10]:
import copy
import mysql.connector
from collections import defaultdict

# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

# Create a cursor object to execute queries
cursor = conn.cursor()

# Query to get Potential Payouts grouped by ParticipantName and EventType
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.EventType IN ('Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

# Execute the query
cursor.execute(query)

# Fetch results and organize them in a dictionary
payouts = defaultdict(lambda: {'payout_conference': 0, 'payout_championship': 0})

print("Potential Payouts from Database:")
for participant_name, event_type, total_payout in cursor.fetchall():
    # Store full team names as keys
    team_name = participant_name
    # Convert the decimal.Decimal to float for compatibility
    total_payout = float(total_payout)
    
    if event_type == 'Conference Winner':
        payouts[team_name]['payout_conference'] = total_payout
    elif event_type == 'Championship':
        payouts[team_name]['payout_championship'] = total_payout
    
    # Print out the potential payout for verification
    print(f"Team: {team_name}, Event Type: {event_type}, Total Payout: {total_payout}")

# Close the cursor and connection
cursor.close()
conn.close()

# Define the probabilities and payouts for each team in the AL and NL
teams_probs_mlb = [
    # AL Teams
    {'team': 'New York Yankees', 'seed': 1, 'conference': 'AL', 'win_conference': 0.34, 'win_championship': 0.17},
    {'team': 'Cleveland Guardians', 'seed': 2, 'conference': 'AL', 'win_conference': 0.30, 'win_championship': 0.15},
    {'team': 'Houston Astros', 'seed': 3, 'conference': 'AL', 'win_conference': 0.16, 'win_championship': 0.08},
    {'team': 'Baltimore Orioles', 'seed': 4, 'conference': 'AL', 'win_conference': 0.1, 'win_championship': 0.06},
    {'team': 'Kansas City Royals', 'seed': 5, 'conference': 'AL', 'win_conference': 0.04, 'win_championship': 0.02},
    {'team': 'Detroit Tigers', 'seed': 6, 'conference': 'AL', 'win_conference': 0.04, 'win_championship': 0.02},
    # NL Teams
    {'team': 'Los Angeles Dodgers', 'seed': 1, 'conference': 'NL', 'win_conference': 0.40, 'win_championship': 0.20},
    {'team': 'Philadelphia Phillies', 'seed': 2, 'conference': 'NL', 'win_conference': 0.30, 'win_championship': 0.15},
    {'team': 'Milwaukee Brewers', 'seed': 3, 'conference': 'NL', 'win_conference': 0.08, 'win_championship': 0.04},
    {'team': 'San Diego Padres', 'seed': 4, 'conference': 'NL', 'win_conference': 0.14, 'win_championship': 0.07},
    {'team': 'Arizona Diamondbacks', 'seed': 5, 'conference': 'NL', 'win_conference': 0.04, 'win_championship': 0.02},
    {'team': 'New York Mets', 'seed': 6, 'conference': 'NL', 'win_conference': 0.04, 'win_championship': 0.02}
]

# Update the teams_probs_mlb dictionary with the payouts from the database
for team in teams_probs_mlb:
    team_name = team['team']
    if team_name in payouts:
        team['payout_conference'] = float(payouts[team_name]['payout_conference'])
        team['payout_championship'] = float(payouts[team_name]['payout_championship'])
    else:
        team['payout_conference'] = 0.0
        team['payout_championship'] = 0.0

# Print the updated teams_probs_mlb to verify
print("\nUpdated Teams Payouts:")
for team in teams_probs_mlb:
    print(team)


Potential Payouts from Database:
Team: New York Jets, Event Type: Conference Winner, Total Payout: 2400.0
Team: New York Jets, Event Type: Championship, Total Payout: 130885.0
Team: Minnesota Twins, Event Type: Conference Winner, Total Payout: 12540.0
Team: Minnesota Twins, Event Type: Championship, Total Payout: 62222.91
Team: Pittsburgh Pirates, Event Type: Conference Winner, Total Payout: 179630.0
Team: Cincinnati Reds, Event Type: Conference Winner, Total Payout: 202905.0
Team: St. Louis Cardinals, Event Type: Championship, Total Payout: 55040.0
Team: Arizona Diamondbacks, Event Type: Conference Winner, Total Payout: 40600.0
Team: St. Louis Cardinals, Event Type: Conference Winner, Total Payout: 10440.0
Team: NY Knicks, Event Type: Championship, Total Payout: 2250.0
Team: New York Knicks, Event Type: Conference Winner, Total Payout: 2400.0
Team: Houston Astros, Event Type: Championship, Total Payout: 62326.0
Team: Philadelphia 76ers, Event Type: Conference Winner, Total Payout: 620

In [8]:
import copy
import mysql.connector
from collections import defaultdict

# Establish a connection to your MySQL database
conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

# Create a cursor object to execute queries
cursor = conn.cursor()

# Query to get Potential Payouts grouped by ParticipantName and EventType
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.EventType IN ('Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

# Execute the query
cursor.execute(query)

# Fetch results and organize them in a dictionary
payouts = defaultdict(lambda: {'payout_conference': 0, 'payout_championship': 0})

# Define the relevant teams for quick lookup
relevant_teams = [
    'New York Yankees', 'Cleveland Guardians', 'Houston Astros', 'Baltimore Orioles',
    'Kansas City Royals', 'Detroit Tigers', 'Los Angeles Dodgers', 'Philadelphia Phillies',
    'Milwaukee Brewers', 'San Diego Padres', 'Arizona Diamondbacks', 'New York Mets'
]

print("Potential Payouts from Database (Filtered by Relevant Teams):")
for participant_name, event_type, total_payout in cursor.fetchall():
    # Check if the participant name matches any relevant team
    if participant_name in relevant_teams:
        # Store full team names as keys
        team_name = participant_name
        # Convert the decimal.Decimal to float for compatibility
        total_payout = float(total_payout)
        
        if event_type == 'Conference Winner':
            payouts[team_name]['payout_conference'] = total_payout
        elif event_type == 'Championship':
            payouts[team_name]['payout_championship'] = total_payout
        
        # Print out the potential payout for verification
        print(f"Team: {team_name}, Event Type: {event_type}, Total Payout: {total_payout}")

# Close the cursor and connection
cursor.close()
conn.close()


Potential Payouts from Database (Filtered by Relevant Teams):
Team: Arizona Diamondbacks, Event Type: Conference Winner, Total Payout: 40600.0
Team: Houston Astros, Event Type: Championship, Total Payout: 62326.0
Team: Houston Astros, Event Type: Conference Winner, Total Payout: 18685.0
Team: Arizona Diamondbacks, Event Type: Championship, Total Payout: 39415.0
Team: Los Angeles Dodgers, Event Type: Conference Winner, Total Payout: 16500.0
Team: San Diego Padres, Event Type: Conference Winner, Total Payout: 16725.0
Team: San Diego Padres, Event Type: Championship, Total Payout: 61080.0
Team: Milwaukee Brewers, Event Type: Championship, Total Payout: 18980.0
Team: Kansas City Royals, Event Type: Conference Winner, Total Payout: 6735.0
Team: Kansas City Royals, Event Type: Championship, Total Payout: 48838.0
Team: Detroit Tigers, Event Type: Conference Winner, Total Payout: 66065.0
Team: New York Mets, Event Type: Championship, Total Payout: 14860.0
Team: Detroit Tigers, Event Type: Cham

In [7]:
import copy

# Define the probabilities and payouts for each team in the AL and NL
teams_probs_mlb = [
    # AL Teams
    {'team': 'Yankees', 'seed': 1, 'conference': 'AL', 'win_conference': 0.34, 'win_championship': 0.17, 'payout_conference': 1802, 'payout_championship': 7200},
    {'team': 'Guardians', 'seed': 2, 'conference': 'AL', 'win_conference': 0.30, 'win_championship': 0.15, 'payout_conference': 5080, 'payout_championship': 0},
    {'team': 'Astros', 'seed': 3, 'conference': 'AL', 'win_conference': 0.16, 'win_championship': 0.08, 'payout_conference': 18685, 'payout_championship': 62326},
    {'team': 'Orioles', 'seed': 4, 'conference': 'AL', 'win_conference': 0.1, 'win_championship': 0.06, 'payout_conference': 8182, 'payout_championship': 6160},
    {'team': 'Royals', 'seed': 5, 'conference': 'AL', 'win_conference': 0.04, 'win_championship': 0.02, 'payout_conference': 6735, 'payout_championship': 48838},
    {'team': 'Tigers', 'seed': 6, 'conference': 'AL', 'win_conference': 0.04, 'win_championship': 0.02, 'payout_conference': 66065, 'payout_championship': 204452},
    # NL Teams
    {'team': 'Dodgers', 'seed': 1, 'conference': 'NL', 'win_conference': 0.40, 'win_championship': 0.20, 'payout_conference': 16500, 'payout_championship': 0},
    {'team': 'Phillies', 'seed': 2, 'conference': 'NL', 'win_conference': 0.30, 'win_championship': 0.15, 'payout_conference': 0, 'payout_championship': 0},
    {'team': 'Brewers', 'seed': 3, 'conference': 'NL', 'win_conference': 0.08, 'win_championship': 0.04, 'payout_conference': 0, 'payout_championship': 18980},
    {'team': 'Padres', 'seed': 4, 'conference': 'NL', 'win_conference': 0.14, 'win_championship': 0.07, 'payout_conference': 16725, 'payout_championship': 61080},
    {'team': 'Diamondbacks', 'seed': 5, 'conference': 'NL', 'win_conference': 0.04, 'win_championship': 0.02, 'payout_conference': 40600, 'payout_championship': 39415},
    {'team': 'Mets', 'seed': 6, 'conference': 'NL', 'win_conference': 0.04, 'win_championship': 0.02, 'payout_conference': 6730, 'payout_championship': 14860}
]

# Define function to update win probabilities based on proximity weights for MLB
def update_win_probs_with_proximity_mlb(winning_team_seed, losing_team_seed, teams_probs, conference):
    """
    Update win probabilities based on the outcome of a specific matchup and proximity in the bracket.
    :param winning_team_seed: Seed of the team that won the matchup.
    :param losing_team_seed: Seed of the team that lost the matchup.
    :param teams_probs: List of dictionaries with each team's probabilities and seed information.
    :param conference: The conference (AL or NL) where the matchup took place.
    :return: Updated probabilities for all remaining teams.
    """
    # Find the losing team's probabilities
    losing_team = next(team for team in teams_probs if team['seed'] == losing_team_seed and team['conference'] == conference)
    
    # Find remaining teams in both conferences
    remaining_teams = [copy.deepcopy(team) for team in teams_probs if not (team['seed'] == losing_team_seed and team['conference'] == conference)]
    
    # Calculate the total remaining probabilities for conference and championship
    total_conference_prob = sum(team['win_conference'] for team in remaining_teams if team['conference'] == conference)
    total_championship_prob = sum(team['win_championship'] for team in remaining_teams)
    
    # Proximity weights
    same_conference_weight = 0.85  # Teams in the same conference as the winner
    opposite_conference_weight = 0.15  # Teams in the opposite conference
    
    for team in remaining_teams:
        # Determine proximity weight
        if team['conference'] == conference:  # Same conference as the matchup
            proximity_weight = same_conference_weight
            
            # Calculate adjusted conference probability (only for same conference teams)
            redistribution_share_conference = (team['win_conference'] / total_conference_prob) * proximity_weight
            team['adjusted_win_conference'] = team['win_conference'] + (redistribution_share_conference * losing_team['win_conference'])
        else:
            team['adjusted_win_conference'] = team['win_conference']
            proximity_weight = opposite_conference_weight

        # Calculate adjusted championship probability (all teams)
        redistribution_share_championship = (team['win_championship'] / total_championship_prob) * proximity_weight
        team['adjusted_win_championship'] = team['win_championship'] + (redistribution_share_championship * losing_team['win_championship'])
    
    return remaining_teams

# Function to calculate total EV for conference and championship bets for MLB
def calculate_total_ev_mlb(remaining_teams):
    """
    Calculate the total expected value for all teams given updated probabilities.
    :param remaining_teams: List of teams with updated probabilities.
    :return: Total expected value of the portfolio.
    """
    ev_conference = sum(team['adjusted_win_conference'] * team['payout_conference'] for team in remaining_teams)
    ev_championship = sum(team['adjusted_win_championship'] * team['payout_championship'] for team in remaining_teams)
    return ev_conference + ev_championship

# Function to calculate EV delta for all first-round matchups for MLB
def calculate_ev_deltas_for_all_matchups_mlb(teams_probs):
    # Define matchups for the wildcard round
    matchups = [
        # AL matchups
        ('AL', 3, 6), ('AL', 4, 5),
        # NL matchups
        ('NL', 3, 6), ('NL', 4, 5)
    ]
    
    ev_deltas = {}
    
    for conference, seed1, seed2 in matchups:
        # Deep copy the original teams_probs for each scenario to avoid data pollution
        original_probs = copy.deepcopy(teams_probs)
        
        # Scenario 1: Higher seed wins
        updated_probs_higher_seed_wins = update_win_probs_with_proximity_mlb(seed1, seed2, original_probs, conference)
        ev_higher_seed_wins = calculate_total_ev_mlb(updated_probs_higher_seed_wins)
        
        # Deep copy the original teams_probs for the second scenario
        original_probs = copy.deepcopy(teams_probs)
        
        # Scenario 2: Lower seed wins
        updated_probs_lower_seed_wins = update_win_probs_with_proximity_mlb(seed2, seed1, original_probs, conference)
        ev_lower_seed_wins = calculate_total_ev_mlb(updated_probs_lower_seed_wins)
        
        # Calculate the EV delta
        ev_delta = abs(ev_higher_seed_wins - ev_lower_seed_wins)
        
        ev_deltas[(conference, seed1, seed2)] = ev_delta
    
    return ev_deltas

# Calculate and output the EV deltas for all first-round matchups for MLB
ev_deltas_mlb = calculate_ev_deltas_for_all_matchups_mlb(teams_probs_mlb)
for matchup, ev_delta in ev_deltas_mlb.items():
    conference, seed1, seed2 = matchup
    print(f"EV Delta for {conference} matchup between Seed {seed1} and Seed {seed2}: {ev_delta}")


EV Delta for AL matchup between Seed 3 and Seed 6: 73.52404205358471
EV Delta for AL matchup between Seed 4 and Seed 5: 990.7581022973973
EV Delta for NL matchup between Seed 3 and Seed 6: 359.8250144188132
EV Delta for NL matchup between Seed 4 and Seed 5: 3234.431382485287
